In [2]:
# !pip install protobuf==3.20.3 --force-reinstall
# !pip install trl
# !pip install bitsandbytes
# # print("completed")

In [4]:
import torch
import os 
from datasets import load_dataset
from transformers import (
AutoModelForCausalLM,
AutoTokenizer,
BitsAndBytesConfig,
HfArgumentParser,
TrainingArguments,
Pipeline,
logging
)

from peft import LoraConfig ,PeftModel
from trl import SFTTrainer

import re
dataset_1 = load_dataset("timdettmers/openassistant-guanaco")
dataset_1

Repo card metadata block was not found. Setting CardData to empty.


DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 9846
    })
    test: Dataset({
        features: ['text'],
        num_rows: 518
    })
})

In [15]:
dataset_1['train'][2]

{'text': '### Human: Can you explain contrastive learning in machine learning in simple terms for someone new to the field of ML?### Assistant: Sure! Let\'s say you want to build a model which can distinguish between images of cats and dogs. You gather your dataset, consisting of many cat and dog pictures. Then you put them through a neural net of your choice, which produces some representation for each image, a sequence of numbers like [0.123, 0.045, 0.334, ...]. The problem is, if your model is unfamiliar with cat and dog images, these representations will be quite random. At one time a cat and a dog picture could have very similar representations (their numbers would be close to each other), while at others two cat images may be represented far apart. In simple terms, the model wouldn\'t be able to tell cats and dogs apart. This is where contrastive learning comes in.\n\nThe point of contrastive learning is to take pairs of samples (in this case images of cats and dogs), then train 

In [4]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU not available, using CPU")


GPU is available: Tesla T4


# bnb configuration

In [5]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,                     # Enable 4-bit weights
    bnb_4bit_quant_type="nf4",             # Normalized Float 4 (best for LLMs)
    bnb_4bit_compute_dtype=torch.float16,  # Compute in FP16 for speed & tensor cores
    bnb_4bit_use_double_quant=True         # Nested quantization to improve accuracy
)
print("Complterd")

Complterd


In [6]:
model_name = "NousResearch/Llama-2-7b-chat-hf"
dataset_name = "mlabonne/guanaco-llama2-1k"
new_model = "Llama-2-7b-chat-finetune"

In [7]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [8]:
# Check the device of the first parameter
first_param = next(model.parameters())
print("First parameter device:", first_param.device)

First parameter device: cuda:0


In [9]:
model.config.use_cache = False
model.config.pretraining_tp = 1

# Llama tokenizer and Lora

In [10]:
# load Llama tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# load LoRA configuration
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [7]:
def transform_conversion(example):
    conversation_text = example['text']
    segments = conversation_text.split("###")

    reformat_segments = []

    # Iterate through segments in pairs (Human / Assistant)
    for i in range(1, len(segments) - 1, 2):
        human_text = segments[i].strip().replace('Human:', '').strip()

        if i + 1 < len(segments):
            assistant_text = segments[i + 1].strip().replace('Assistant:', '').strip()
            reformat_segments.append(f"<s>[INST] {human_text} [/INST] {assistant_text} </s>")
        else:
            reformat_segments.append(f"<s>[INST] {human_text} [/INST]</s>")

    return {'text': ''.join(reformat_segments)}


In [5]:
# QLoRA Parameters 
lora_r = 64
lora_alpha = 16
lora_dropout = 0.1

# Quantization
use_4bit = True
bnb_compute_dtype = "float16"          # ✅ corrected name
bnb_4bit_quant_type = "nf4"
use_nested_quant = False               # ✅ corrected spelling

# Training configuration
output_dir = "./qlora-output"          # ✅ should not be empty
num_train_epochs = 1                   # ✅ corrected plural (used by HF Trainer)
fp16 = False
bf16 = False
per_device_train_batch_size = 1
per_device_eval_batch_size = 1

gradient_accumulation_steps = 2        # ✅ corrected spelling
gradient_checkpointing = True          # ✅ corrected spelling
max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"

lr_scheduler_type = "cosine"
max_steps = -1 
warmup_ratio = 0.03
group_by_length = True
save_steps = 0 
logging_steps = 25

# SFT parameters 
max_seq_length = 512
packing = False
device_map = {"": 0}                   # ✅ keep empty key if using single GPU


# TrainingArguments

In [14]:
from transformers import TrainingArguments

training_arguments = TrainingArguments(
    output_dir="./qlora-output",                 # Where checkpoints and logs are saved
    num_train_epochs=1,     # Total epochs
    per_device_train_batch_size=1,         # ✅ T4 16GB + 4-bit → usually batch_size=1 or 2
    per_device_eval_batch_size=1,          # Evaluation batch size
    gradient_accumulation_steps=4,         # Accumulate grads to simulate larger batch
    optim="adamw_torch",                   # Optimizer, works well with small GPUs
    save_steps=200,                        # Save checkpoint every N steps
    logging_steps=50,                      

    learning_rate=2e-4,                     # Adjust for LoRA fine-tuning
    weight_decay=0.0,                      

    fp16=True,                              # Mixed precision for Tesla T4
    max_grad_norm=1.0,                      
    max_steps=-1,                           # Use max_steps=-1 if you want full epochs
    warmup_ratio=0.03,                      

    group_by_length=True,                   # Group sequences by length → saves memory
    lr_scheduler_type="cosine",             # Cosine scheduler works well with LoRA

    report_to="tensorboard",                # TensorBoard logging
    ddp_find_unused_parameters=False        # Not using multi-GPU, safe for single GPU
)


In [22]:
from peft import PeftModel, get_peft_model
from transformers import default_data_collator
from trl import SFTTrainer

# 1️⃣ Attach LoRA (only once)
if not isinstance(model, PeftModel):
    model = get_peft_model(model, peft_config)
    print("LoRA adapter applied.")
else:
    print("Model already has LoRA applied.")

# 2️⃣ Ensure training args are set for 4-bit
training_arguments.fp16 = True   # required for 4-bit
training_arguments.bf16 = False

# 3️⃣ Attach tokenizer to model so TRL can use it
model.tokenizer = tokenizer

Model already has LoRA applied.


In [23]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (

In [24]:
from torch.nn.utils.rnn import pad_sequence
import torch

def data_collator_with_labels(batch):
    # batch is a list of dicts like {'input_ids': [...]}
    input_ids = [torch.tensor(example['input_ids']) for example in batch]
    
    # pad to max length in this batch
    input_ids = pad_sequence(input_ids, batch_first=True, padding_value=0)
    
    # labels are the same as input_ids for causal LM
    labels = input_ids.clone()
    
    return {'input_ids': input_ids, 'labels': labels, 'attention_mask': (input_ids != 0).long()}


In [25]:
new_dataset = dataset_1
new_dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 9846
    })
    test: Dataset({
        features: ['text'],
        num_rows: 518
    })
})

# SFT trainer

In [26]:
trainer = SFTTrainer(
    model=model,
    train_dataset=new_dataset['train'],
    args=training_arguments,
    data_collator=data_collator_with_labels,
)

In [27]:
import torch
print(torch.cuda.get_device_name(0))
print(torch.version.cuda)


Tesla T4
12.4


# trainig is done 

In [28]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 0}.


Step,Training Loss
50,1.683700
100,1.329800
150,1.323900
200,1.277000
250,1.298300
300,1.378400
350,1.283500
400,1.286700
450,1.353200
500,1.302800


TrainOutput(global_step=2462, training_loss=1.31182559034284, metrics={'train_runtime': 11075.7092, 'train_samples_per_second': 0.889, 'train_steps_per_second': 0.222, 'total_flos': 1.6359454553874432e+17, 'train_loss': 1.31182559034284, 'entropy': 1.4191851758438607, 'num_tokens': 4123966.0, 'mean_token_accuracy': 0.6710686845623929, 'epoch': 1.0})

In [29]:
model.save_pretrained("/kaggle/working/qlora_model")
tokenizer.save_pretrained("/kaggle/working/qlora_model")


('/kaggle/working/qlora_model/tokenizer_config.json',
 '/kaggle/working/qlora_model/special_tokens_map.json',
 '/kaggle/working/qlora_model/tokenizer.model',
 '/kaggle/working/qlora_model/added_tokens.json',
 '/kaggle/working/qlora_model/tokenizer.json')

In [30]:
!zip -r /kaggle/working/qlora_model.zip /kaggle/working/qlora_model


  adding: kaggle/working/qlora_model/ (stored 0%)
  adding: kaggle/working/qlora_model/added_tokens.json (stored 0%)
  adding: kaggle/working/qlora_model/adapter_config.json (deflated 55%)
  adding: kaggle/working/qlora_model/tokenizer.json (deflated 85%)
  adding: kaggle/working/qlora_model/README.md (deflated 65%)
  adding: kaggle/working/qlora_model/tokenizer.model

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


 (deflated 55%)
  adding: kaggle/working/qlora_model/tokenizer_config.json (deflated 72%)
  adding: kaggle/working/qlora_model/adapter_model.safetensors (deflated 7%)
  adding: kaggle/working/qlora_model/special_tokens_map.json (deflated 72%)


# load the model and weights  and check the inference rate 

In [5]:
from transformers import AutoTokenizer , AutoModelForCausalLM
from peft import PeftModel
base_model = "NousResearch/Llama-2-7b-chat-hf"
lora_path = "/kaggle/input/tuned-model/kaggle/working/qlora_model"


tokenizer = AutoTokenizer.from_pretrained(lora_path)

model= AutoModelForCausalLM.from_pretrained(
    base_model ,
    torch_dtype = "auto",
    device_map= "auto"
)

model = PeftModel.from_pretrained(model, lora_path)
model.eval()




config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj):

# new Prompting 

In [16]:
prompt = "Can you explain contrastive learning in machine learning in simple terms for someone new to the field of ML?"

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    output = model.generate(
        **inputs,
        max_new_tokens=200,
        temperature=0.7
    )

print(tokenizer.decode(output[0], skip_special_tokens=True))


Can you explain contrastive learning in machine learning in simple terms for someone new to the field of ML?
1. What is contrastive learning?
2. How does contrastive learning work?
3. What are the benefits of contrastive learning?
4. What are the challenges of contrastive learning?
5. How does contrastive learning compare to other machine learning methods?
6. What are some examples of contrastive learning in action?
7. How can I get started with contrastive learning?

Contrastive learning is a type of machine learning that involves training a model to distinguish between two different types of data. The model is trained to recognize the differences between the two types of data, and to use those differences to make predictions or classify new data.

In contrastive learning, the model is trained on a dataset that contains both positive and negative examples. The positive examples are similar to each other, while the negative examples are dissimilar. For example, if the model is trained 